In [1]:
import warnings
warnings.filterwarnings('ignore')

This notebook uses scripts from https://github.com/martin-1992/stacking 

In [3]:
import numpy as np
import xgboost as xgb
from process import coreProcess
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from clfParams import paramDict


class baseClassifier(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, X_train, y_train):
        self.clf.fit(X_train, y_train)

    def predict(self, X):
        return self.clf.predict(X)


##############################################################
def get_oof(clf, X_train, y_train, X_test):
    oof_train = np.zeros((n_train, ))
    oof_test = np.zeros((n_test, ))
    # n_folds * n_test,
    oof_test_skf = np.empty((n_folds, n_test))

    for i, (train_idx, val_idx) in enumerate(kf):
        kf_x_train = X_train[train_idx]
        kf_y_train = y_train[train_idx]
        kf_x_val = X_train[val_idx]

        clf.train(kf_x_train, kf_y_train)
        oof_train[val_idx] = clf.predict(kf_x_val)
        oof_test_skf[i, :] = clf.predict(X_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


def accuracy(pred_y, true_y):
    return np.sum((pred_y == y_train).astype(float)) / len(true_y)


df, train_idx, test_idx = coreProcess()

n_train = len(train_idx)
n_test = len(test_idx)
seed = 0
n_folds = 5
kf = KFold(n_train, n_folds=n_folds, random_state=seed)

param_dict = paramDict()

rf = baseClassifier(clf=RandomForestClassifier, seed=seed, params=param_dict['rf_params'])
et = baseClassifier(clf=ExtraTreesClassifier, seed=seed, params=param_dict['et_params'])
ada = baseClassifier(clf=AdaBoostClassifier, seed=seed, params=param_dict['ada_params'])
gb = baseClassifier(clf=GradientBoostingClassifier, seed=seed, params=param_dict['gb_params'])
svc = baseClassifier(clf=SVC, seed=seed, params=param_dict['svc_params'])

feats_not_use = ['Name', 'Cabin', 'PassengerId', 'Ticket',
                 'Ticket_prefix', 'Survived']
feats_in_use = [col for col in df.columns if col not in feats_not_use]

use_df = df[feats_in_use]
y_train = df.loc[train_idx, 'Survived'].ravel()
X_train = use_df.loc[train_idx, :]
X_train = X_train.values
X_test = use_df.loc[test_idx, :].values

et_oof_train, et_oof_test = get_oof(et, X_train, y_train, X_test)
rf_oof_train, rf_oof_test = get_oof(rf, X_train, y_train, X_test)
ada_oof_train, ada_oof_test = get_oof(ada, X_train, y_train, X_test)
gb_oof_train, gb_oof_test = get_oof(gb, X_train, y_train, X_test)
svc_oof_train, svc_oof_test = get_oof(svc, X_train, y_train, X_test)

level1_train = np.concatenate((et_oof_train, rf_oof_train, ada_oof_train,
                               gb_oof_train, svc_oof_train), axis=1)
level1_test = np.concatenate((et_oof_test, rf_oof_test, ada_oof_test,
                              gb_oof_test, svc_oof_test), axis=1)

clf = xgb.XGBClassifier(
            n_estimators = 2000,
            max_depth = 4,
            min_child_weight = 2,
            gamma = 0.9,
            subsample = 0.8,
            colsample_bytree = 0.8,
            objective = 'binary:logistic',
            nthread = -1,
            scale_pos_weight = 1)

clf.fit(level1_train, y_train)
pred_y = clf.predict(level1_test)


accuracy(clf.predict(level1_train), y_train)

/home/arish/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/arish/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8742985409652076